# <u>**Streaming Service Comparison**</u>

### **Objective:**
- Determine which streaming platform hosts the majority of content I enjoy so that I can pare down the services to which I subscribe. 

### **Data Sources:**
- [Netflix via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-netflix-dataset)
- [Hulu via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-hulu-dataset)
- [Prime via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-amazon-prime-dataset/data)
- [AppleTV via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-apple-tv-dataset)

## **Data Collection & Loading**

### **Import Pandas, Numpy, Matplotlib, Wordcloud, and PIL**

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wordcloud as wc
from PIL import Image

### **Data Load**

In [2]:
# Load in each file separately. 
# Plan to create a function for this 
apple = pd.read_csv("AppleTV.csv")
hulu = pd.read_csv("Hulu.csv")
netflix = pd.read_csv("Netflix.csv")
prime = pd.read_csv("Prime.csv")

In [3]:
display(apple)

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,113403.0,NaN
1,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2348885.0,NaN
2,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1238903.0,NaN
3,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,477481.0,NaN
4,Metropolis,movie,"Drama, Sci-Fi",1927.0,tt0017136,8.3,192285.0,NaN
...,...,...,...,...,...,...,...,...
18203,They Took Our Child: We Got Her Back,tv,"Crime, Documentary",2015.0,tt5056408,8.6,31.0,NaN
18204,NaN,tv,NaN,NaN,NaN,NaN,NaN,NaN
18205,NaN,tv,NaN,2025.0,NaN,NaN,NaN,NaN
18206,NaN,tv,Comedy,2018.0,NaN,NaN,NaN,NaN


In [4]:
# Dispaly information about the dataset (i.e., row counts, column counts, column names, datatypes, # of non-null rows)
apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18208 entries, 0 to 18207
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               17643 non-null  object 
 1   type                18208 non-null  object 
 2   genres              17549 non-null  object 
 3   releaseYear         18179 non-null  float64
 4   imdbId              16727 non-null  object 
 5   imdbAverageRating   16314 non-null  float64
 6   imdbNumVotes        16314 non-null  float64
 7   availableCountries  84 non-null     object 
dtypes: float64(3), object(5)
memory usage: 1.1+ MB


In [5]:
# Drop availableCountries Column - no value with such a low non-null number
apple.drop("availableCountries",axis=1,inplace=True)
apple.columns

Index(['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating',
       'imdbNumVotes'],
      dtype='object')

In [6]:
# Change releaseYear,  imdbNumVotes to integers 
# Display only the data types

apple[["releaseYear","imdbNumVotes"]] = apple[["releaseYear","imdbNumVotes"]].apply(np.int64)
display(apple.dtypes)

c:\Users\rkynhoff\OneDrive - University of Rochester\Documents\CodeLouisville\Streaming_Service_Comparisons\venv\Lib\site-packages\pandas\core\series.py:1031: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
dtype: object

In [7]:
# Rename Columns
apple.rename(columns={"title":"Title","type":"Type","genres":"Genres","releaseYear":"Release Year","imdbId":"IMDb ID","imdbAverageRating":"IMDb Average Rating","imdbNumVotes":"IMDb Num Votes"},inplace = True)
apple.columns

Index(['Title', 'Type', 'Genres', 'Release Year', 'IMDb ID',
       'IMDb Average Rating', 'IMDb Num Votes'],
      dtype='object')

In [ ]:
# Determine how many titles contain null values
apple["Title"].isna().sum()

np.int64(565)

In [29]:
# Drop the rows contiaining null values in the Title column
apple = apple.dropna(subset=["Title"])
apple["Title"].isna().sum()


np.int64(0)

In [30]:
#apple.genres.value_counts()
# apple['genres'].str.split(',', expand=True)
apple[["Genre 1","Genre 2","Genre 3","Genre 4","Genre 5","Genre 6"]] = apple["Genres"].str.split(',',expand=True)

display(apple)



,Title,Type,Genres,Release Year,IMDb ID,IMDb Average Rating,IMDb Num Votes,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Genre 6
0,Four Rooms,movie,Comedy,1995,tt0113101,6.7,113403,Comedy,None,None,None,None,None
1,Forrest Gump,movie,"Drama, Romance",1994,tt0109830,8.8,2348885,Drama,Romance,None,None,None,None
2,American Beauty,movie,Drama,1999,tt0169547,8.3,1238903,Drama,None,None,None,None,None
3,Citizen Kane,movie,"Drama, Mystery",1941,tt0033467,8.3,477481,Drama,Mystery,None,None,None,None
4,Metropolis,movie,"Drama, Sci-Fi",1927,tt0017136,8.3,192285,Drama,Sci-Fi,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18188,Das Boot - Die komplette TV-Serie,tv,"Drama, War",1985,tt30970892,8.8,198,Drama,War,None,None,None,None
18194,The Amazon Review Killer,tv,Documentary,2024,tt33342317,6.4,64,Documentary,None,None,None,None,None
18198,Conspirators,tv,Mystery,2025,tt35075008,5.5,44,Mystery,None,None,None,None,None
18200,Extraordinary World,tv,Documentary,2025,tt34888944,NaN,-9223372036854775808,Documentary,None,None,None,None,None


In [9]:
# View top 5 rows in the hulu df
hulu.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8949.0,NaN
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986.0,tt0092149,7.5,7727.0,NaN
2,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2348885.0,NaN
3,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,521815.0,NaN
4,My Life Without Me,movie,"Drama, Romance",2003.0,tt0314412,7.4,26140.0,NaN


In [10]:
# Add a column to hulu DF and entered a default value for each row & display bottom two rows
hulu["serviceName"] = "Hulu"
hulu.tail(2)

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,serviceName
10201,Cardcaptor Sakura: Clear Card Arc,tv,"Adventure, Animation, Comedy",2018.0,tt6279576,7.7,879.0,NaN,Hulu
10202,Faces of Music,tv,Documentary,2025.0,tt35460841,NaN,NaN,NaN,Hulu


In [11]:
# View top 5 rows in the netflix df
netflix.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1238903.0,NaN
1,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,521815.0,NaN
2,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1236086.0,NaN
3,Jarhead,movie,"Biography, Drama, War",2005.0,tt0418763,7.0,213667.0,NaN
4,Unforgiven,movie,"Drama, Western",1992.0,tt0105695,8.2,448833.0,NaN


In [12]:
# Add a column to netflix DF and entered a default value for each row & display bottom two rows
netflix["serviceName"] = "Netflix"
netflix.tail(2)

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,serviceName
20734,American Manhunt: O.J. Simpson,tv,Documentary,2025.0,tt35456246,7.4,1148.0,NaN,Netflix
20735,Devil's Diner,tv,"Drama, Horror",2025.0,tt35557166,7.2,289.0,NaN,Netflix


In [13]:
# View top 5 rows of prime df
prime.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8949.0,NaN
1,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,113403.0,NaN
2,Judgment Night,movie,"Action, Crime, Drama",1993.0,tt0107286,6.6,19627.0,NaN
3,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2348885.0,NaN
4,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,477481.0,NaN


In [14]:
# Add a column to the prime df and entered a default value for each row & display bottom two rows
prime["serviceName"] = "Prime"
prime.tail(2)

# map function for creating column and populating each cell within each df with the df name

# create a dictionary with the list of the names of df and have it look at the df name to populate the column

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,serviceName
70053,Ryan Trahan,tv,Reality-TV,2019.0,tt28487606,8.2,42.0,NaN,Prime
70054,Sivarapalli,tv,Comedy,2025.0,tt31914057,7.4,120.0,NaN,Prime


In [15]:
# Merge the dataframes to append them to the end of each other since using the same column names
streaming = pd.concat([apple, hulu, netflix, prime], axis = 0)


In [16]:
streaming.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119202 entries, 0 to 70054
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Title                17643 non-null   object 
 1   Type                 18208 non-null   object 
 2   Genres               17549 non-null   object 
 3   Release Year         18208 non-null   float64
 4   IMDb ID              16727 non-null   object 
 5   IMDb Average Rating  16314 non-null   float64
 6   IMDb Num Votes       18208 non-null   float64
 7   Genre 1              17549 non-null   object 
 8   Genre 2              11705 non-null   object 
 9   Genre 3              6670 non-null    object 
 10  Genre 4              23 non-null      object 
 11  Genre 5              3 non-null       object 
 12  Genre 6              1 non-null       object 
 13  title                98045 non-null   object 
 14  type                 100994 non-null  object 
 15  genres               98

In [17]:
streaming.describe()

,Release Year,IMDb Average Rating,IMDb Num Votes,releaseYear,imdbAverageRating,imdbNumVotes
count,1.820800e+04,16314.000000,1.820800e+04,100736.000000,89195.000000,8.919500e+04
mean,-1.469012e+16,6.381574,-9.594171e+17,2009.132951,6.110641,1.936937e+04
std,3.678099e+17,1.162677,2.815851e+18,17.791458,1.308682,9.616132e+04
min,-9.223372e+18,1.300000,-9.223372e+18,1902.000000,1.000000,5.000000e+00
25%,2.000000e+03,5.700000,9.000000e+01,2006.000000,5.300000,1.290000e+02
50%,2.014000e+03,6.500000,8.470000e+02,2016.000000,6.300000,6.160000e+02
75%,2.020000e+03,7.200000,6.243250e+03,2020.000000,7.000000,3.575000e+03
max,2.025000e+03,9.600000,2.348885e+06,2026.000000,10.000000,3.002274e+06


In [18]:
streaming.duplicated().value_counts()

False    116987
True       2215
Name: count, dtype: int64

In [19]:
streaming[streaming.duplicated(keep=False)]

,Title,Type,Genres,Release Year,IMDb ID,IMDb Average Rating,IMDb Num Votes,Genre 1,Genre 2,Genre 3,...,Genre 6,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,serviceName
3497,A Personal Journey with Martin Scorsese Throug...,movie,"Biography, Documentary, History",1995.0,tt0112120,8.5,5.196000e+03,Biography,Documentary,History,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13091,The Kidnap & Murder of Lynda Spence,movie,"Documentary, Crime",2023.0,NaN,NaN,-9.223372e+18,Documentary,Crime,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13093,The Kidnap & Murder of Lynda Spence,movie,"Documentary, Crime",2023.0,NaN,NaN,-9.223372e+18,Documentary,Crime,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13980,A Personal Journey with Martin Scorsese Throug...,movie,"Biography, Documentary, History",1995.0,tt0112120,8.5,5.196000e+03,Biography,Documentary,History,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,NaN,tv,NaN,2007.0,NaN,NaN,-9.223372e+18,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,tv,NaN,2021.0,NaN,NaN,NaN,NaN,Prime
70040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,tv,NaN,NaN,NaN,NaN,NaN,NaN,Prime
70041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,tv,Documentary,2025.0,NaN,NaN,NaN,NaN,Prime
70047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,tv,NaN,2024.0,NaN,NaN,NaN,NaN,Prime


In [20]:
streaming["title"].isnull().value_counts()


title
False    98045
True     21157
Name: count, dtype: int64

In [21]:
streaming["imdbId"].isnull().value_counts()

imdbId
False    92024
True     27178
Name: count, dtype: int64

In [22]:
# # Assign dataset names & combine to read in as separate data frames

# list_of_names = ['Netflix','Hulu','Prime','AppleTV']

# # Created an empty list into which I can place the datasets
# combined_list = []

# # Used a function to append the datasets into the empty list I created above
# for i in range(len(list_of_names)):
#     temp_df = pd.read_csv(list_of_names[i]+".csv")
#     combined_list.append(temp_df)

### **Initial Checks**

In [23]:
# # Looking at file content of list in index 0 - Netflix - to determine column names, non-null counts, datatypes, and what the top five rows look like
# combined_list[0].info()
# combined_list[0].head(5)



In [24]:
# # Looking at file content of list in index 1 - Hulu - to determine column names, non-null counts, datatypes, and what the top five rows look like
# combined_list[1].info()
# combined_list[1].head(5)

In [25]:
# # Looking at file content of list in index 2 - Prime - to determine column names, non-null counts, datatypes, and what the top five rows look like
# combined_list[2].info()
# combined_list[2].head(5)

In [26]:
# # Looking at file content of list in index 3 - AppleTV - to determine column names, non-null counts, datatypes, and what the top five rows look like
# combined_list[3].info()
# combined_list[3].head(5)

### Initiall Review (02/10/2025)
- The "availableCountries" column will not provide much data going forward
- No dataset has a column for the source of the data, so this will need to be added

### Secondary Review (02/12/2025)
- Each dataset contains an "imdbId" column with the listing's IMDb ID, which is a string; this may come in handy 
- There are 3,514 missing titles
    - Prime = 2234
    - AppleTV = 651
    - Hulu = 2885
    - Netflix = 629
- There are 2,590 duplicated rows, among which many are from missing titles, though several are duplicate title names 